In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
from tqdm import tqdm

In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [9]:
#links e classes
VIVA_REAL_LINK = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/'
RESULT_LIST_CLASS = 'results-list'
ADDRESS_CLASS = 'property-card__address'
AREA_CLASS = 'property-card__detail-area'
ROOM_CLASS = 'property-card__detail-room'
BATHROOM_CLASS = 'property-card__detail-bathroom'
GARAGE_CLASS = 'property-card__detail-garage'
AMENITIES_CLASS = 'property-card__amenities'
PRICE_CLASS = 'property-card__price'
CONDO_CLASS = 'js-condo-price'
VALUE_CLASS = 'property-card__detail-value'
CHANGE_PAGE_XPATH = "//a[@title='Próxima página']"

In [13]:
driver.get(VIVA_REAL_LINK)
full_results = []

num_pages = 200

for page in tqdm(range(num_pages)):
  #sleep necessario para garantir o carregamento da pagina
  sleep(6)

  #verificar se nao tem pop-up de cookie para aceitar
  try:
    driver.find_element(By.CLASS_NAME, "cookie-notifier__cta").click()
  except:
    print("No cookies!")

  #busca a lista de todos os resultados da pagina
  results_page = driver.find_element(By.CLASS_NAME,RESULT_LIST_CLASS)
  #cria uma lista com todos os elementos que estao dentro de uma tag div e possuem o atributo data-type
  ids_list = results_page.find_elements_by_xpath('//div[@data-type]')
  for j in range(len(ids_list)):

    #coleta o id do ap
    url = ids_list[j].find_element(By.CLASS_NAME,'js-card-title').get_attribute('href')
    id = url.split("id-")[-1][:-1]

    #essa validacao existe para pular o card de anuncio que existe
    #caso seja ap, captura o endereco
    try:
      address = ids_list[j].find_element(By.CLASS_NAME,ADDRESS_CLASS).text
    except:
      continue

    #recebe o elemento onde tem a informacao da area
    area_class = ids_list[j].find_element(By.CLASS_NAME,AREA_CLASS)
    #captura apenas o numero da area
    area = area_class.find_element(By.CLASS_NAME,VALUE_CLASS).text

    #recebe o elemento onde tem a informacao de quartos
    room_class = ids_list[j].find_element(By.CLASS_NAME,ROOM_CLASS)
    #captura apenas o numero de quartos
    room = room_class.find_element(By.CLASS_NAME,VALUE_CLASS).text

    #recebe o elemento onde tem a informacao de banheiros
    bathroom_class = ids_list[j].find_element(By.CLASS_NAME,BATHROOM_CLASS)
    #captura apenas o numero de banheiros
    bathroom = bathroom_class.find_element(By.CLASS_NAME,VALUE_CLASS).text

    #recebe o elemento onde tem a informacao de vagas
    garage_class = ids_list[j].find_element(By.CLASS_NAME,GARAGE_CLASS)
    #captura apenas o numero de vagas
    garage = garage_class.find_element(By.CLASS_NAME,VALUE_CLASS).text

    #verifica se existem extras e captura
    try:
      amenities = ids_list[j].find_element(By.CLASS_NAME,AMENITIES_CLASS).text
    except:
      amenities = None

    #recebe o elemento onde tem a informacao do preco do aluguel
    price_class = ids_list[j].find_element(By.CLASS_NAME,PRICE_CLASS)
    #captura o valor do aluguel
    price = price_class.find_element_by_tag_name('p').text

    #verifica se existe valor de condominio e captura
    try:
      condo = ids_list[j].find_element(By.CLASS_NAME,CONDO_CLASS).text
    except:
      condo = None

    #adiciona um dicionario com todas as informacoes do ap na lista final
    full_results.append({'id': id,
                         'address': address,
                         'area': area,
                         'room': room,
                         'bathroom': bathroom,
                         'garage': garage,
                         'amenities': amenities,
                         'price': price,
                         'condo': condo})

  #recebe o elemento onde tem a troca de página e clica
  next_page = driver.find_element(By.XPATH, CHANGE_PAGE_XPATH)
  #print(next_page)
  next_page.click()

df = pd.DataFrame(full_results)
df.to_csv('full_results.csv', index=False)
driver.close()

  0%|          | 0/200 [00:06<?, ?it/s]

No cookies!


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".results-list"}
  (Session info: chrome=135.0.7049.96); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7FB675305+78597]
	GetHandleVerifier [0x00007FF7FB675360+78688]
	(No symbol) [0x00007FF7FB4291AA]
	(No symbol) [0x00007FF7FB47F149]
	(No symbol) [0x00007FF7FB47F3FC]
	(No symbol) [0x00007FF7FB4D2467]
	(No symbol) [0x00007FF7FB4A712F]
	(No symbol) [0x00007FF7FB4CF2BB]
	(No symbol) [0x00007FF7FB4A6EC3]
	(No symbol) [0x00007FF7FB4703F8]
	(No symbol) [0x00007FF7FB471163]
	GetHandleVerifier [0x00007FF7FB91EEBD+2870973]
	GetHandleVerifier [0x00007FF7FB919668+2848360]
	GetHandleVerifier [0x00007FF7FB936943+2967875]
	GetHandleVerifier [0x00007FF7FB69014A+188746]
	GetHandleVerifier [0x00007FF7FB69842F+222255]
	GetHandleVerifier [0x00007FF7FB67D284